In [1]:
from dataloader import DataLoader
import pandas as pd
import numpy as np

In [2]:
loader = DataLoader()

In [3]:
loader.load_sells_data_from_file('sprzedaz.csv')
df_filtered = loader = loader.prepare_sells_data()


In [4]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

# 2. Lista kolumn miesięcznych: od 96 do 1 (jako stringi)
miesiace_kolumny = [str(i) for i in range(96, 0, -1)]

# 3. Agregacja: suma sprzedaży na poziomie KodProduktu
df_agg = df_filtered.groupby("KodProduktu")[miesiace_kolumny].sum().reset_index()

# 4. Long format
df_long = df_agg.melt(id_vars="KodProduktu", var_name="miesiac_ago", value_name="sprzedaz")
df_long["miesiac_ago"] = df_long["miesiac_ago"].astype(int)
df_long["czas"] = 97 - df_long["miesiac_ago"]  # np. 1 to najstarszy, 96 to ostatni znany

# 5. Tworzenie cech lagowych
lags = [1, 2, 3]
for lag in lags:
    df_long[f"lag_{lag}"] = df_long.groupby("KodProduktu")["sprzedaz"].shift(lag)

# 6. Usunięcie braków
df_lags = df_long.dropna().copy()

# 7. Zakodowanie KodProduktu
le = LabelEncoder()
df_lags["KodProduktu_enc"] = le.fit_transform(df_lags["KodProduktu"])

# 8. Przygotowanie danych treningowych
X = df_lags[[f"lag_{l}" for l in lags] + ["KodProduktu_enc"]]
y = df_lags["sprzedaz"]

# 9. Trening modelu
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)

# 10. Predykcja dla wybranego produktu
kod = "0014"  # <- tutaj wpisz KodProduktu, dla którego chcesz prognozę

# Ostatnie dane tego produktu
df_kod = df_lags[df_lags["KodProduktu"] == kod].sort_values("czas", ascending=False)

# Bierzemy ostatnie znane lags
last_values = df_kod.head(3)["sprzedaz"].tolist()

# Zakoduj produkt
kod_enc = le.transform([kod])[0]

# Predykcja 3 przyszłych miesięcy: 0, -1, -2
preds = []
for _ in range(3):
    x_input = [last_values[-1], last_values[-2], last_values[-3]][::-1]
    x_input.append(kod_enc)
    y_pred = model.predict([x_input])[0]
    preds.append(y_pred)
    last_values.append(y_pred)

# 11. Wynik
for i, p in enumerate(preds):
    print(f"Prognozowana sprzedaż produktu {kod} za miesiąc {i} od teraz: {p:.2f}")

Prognozowana sprzedaż produktu 0014 za miesiąc 0 od teraz: 0.00
Prognozowana sprzedaż produktu 0014 za miesiąc 1 od teraz: 0.00
Prognozowana sprzedaż produktu 0014 za miesiąc 2 od teraz: 0.00


c:\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [8]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

# 2. Kolumny miesięczne
miesiace_kolumny = [str(i) for i in range(96, 0, -1)]

# 3. Agregacja do poziomu produktu
df_agg = df_filtered.groupby("KodProduktu")[miesiace_kolumny].sum().reset_index()

# 4. Przekształcenie na długi format
df_long = df_agg.melt(id_vars="KodProduktu", var_name="miesiac_ago", value_name="sprzedaz")
df_long["miesiac_ago"] = df_long["miesiac_ago"].astype(int)
df_long["czas"] = 97 - df_long["miesiac_ago"]

# 5. Tworzenie lagów
lags = [1, 2, 3]
for lag in lags:
    df_long[f"lag_{lag}"] = df_long.groupby("KodProduktu")["sprzedaz"].shift(lag)

df_lags = df_long.dropna().copy()

# 6. Kodowanie KodProduktu
le = LabelEncoder()
df_lags["KodProduktu_enc"] = le.fit_transform(df_lags["KodProduktu"])

# 7. Dane treningowe
X = df_lags[[f"lag_{l}" for l in lags] + ["KodProduktu_enc"]]
y = df_lags["sprzedaz"]

# 8. Trening modelu XGBoost
model = xgb.XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42)
model.fit(X, y)

# 9. Prognozowanie dla wybranego produktu
kod = "0003"  # <- wpisz dowolny KodProduktu
kod_enc = le.transform([kod])[0]

# Ostatnie znane dane produktu
df_kod = df_lags[df_lags["KodProduktu"] == kod].sort_values("czas", ascending=False)
last_values = df_kod.head(3)["sprzedaz"].tolist()

# Prognozujemy na 3 przyszłe miesiące
preds = []
for _ in range(3):
    x_input = [last_values[-1], last_values[-2], last_values[-3]][::-1]
    x_input.append(kod_enc)
    y_pred = model.predict([x_input])[0]
    preds.append(y_pred)
    last_values.append(y_pred)

# 10. Wynik
for i, p in enumerate(preds):
    print(f"Prognozowana sprzedaż produktu {kod} za miesiąc {i} od teraz: {p:.2f}")

Prognozowana sprzedaż produktu 0003 za miesiąc 0 od teraz: 37.47
Prognozowana sprzedaż produktu 0003 za miesiąc 1 od teraz: 9.64
Prognozowana sprzedaż produktu 0003 za miesiąc 2 od teraz: 32.32


ValueError: With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.